# BayesDB analysis of fish bouts in relation to paramecia characteristics

## 1. Ingesting the data

### a. Launching BayesDB

In [1]:
from csv_analysis import invert_all_bouts
import pandas as pd
#drct = '042318_6/'
drct = 'wik_bdb/'
#csv_file = drct + 'all_huntbouts_rev.csv'
#data = pd.read_csv(csv_file)
#invert_all_bouts(data, drct)


In [2]:
%load_ext jupyter_probcomp.magics

session_id: jovyan@nightcrawler2-notebook_2019-03-21T15:43:14.919028_3


In [3]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

In [4]:
#!rm -f wik_bdb/bdb_hunts_inverted.bdb
#%bayesdb -j wik_bdb/bdb_hunts_inverted.bdb

!rm -f wik_bdb/bdb_hunts_marr.bdb
%bayesdb -j wik_bdb/bdb_hunts_marr.bdb


u'Loaded: wik_bdb/bdb_hunts_marr.bdb'

### b. Loading and printing the data from a `.csv` file

In [5]:
%sql DROP TABLE IF EXISTS bout_table;
#%bql CREATE TABLE bout_table FROM 'wik_bdb/huntbouts_inverted.csv' 
%bql CREATE TABLE bout_table FROM 'wik_bdb/all_huntbouts_marr.csv' 

""


In [6]:
%sql SELECT * FROM bout_table LIMIT 25;

,Bout Number,Para Az,Para Alt,Para Dist,Postbout Para Az,Postbout Para Alt,Postbout Para Dist,Strike Or Abort
0,0.0,-0.977032,0.629180,354.291281,-0.175346,0.592776,299.960989,2.0
1,1.0,-0.162832,0.654781,305.528267,-0.158165,0.736650,286.473525,2.0
2,2.0,-0.179093,0.818221,281.527199,0.171628,0.271518,200.860390,2.0
3,3.0,0.073963,0.210280,134.098036,0.012218,0.210299,95.292186,2.0
4,4.0,0.108441,0.299536,77.905007,0.221962,0.746375,60.931606,2.0
5,0.0,-0.442195,0.487576,78.876586,0.053742,0.512896,63.664664,1.0
6,0.0,0.461436,0.400363,372.395718,0.189287,0.488262,290.246365,1.0
7,1.0,0.198364,0.499294,282.973273,-0.067488,0.434303,188.771498,1.0
8,2.0,-0.119681,0.392236,171.041294,0.027759,0.181165,106.936309,1.0
9,3.0,0.039726,0.164327,102.885812,0.118466,0.163395,76.723493,1.0


In [7]:
%bql .nullify bout_table ''
%bql .nullify bout_table 'nan'

Nullified 0 cells
Nullified 156 cells


## 2. Automatically learning a CrossCat probabilistic model

### a. Defining an analysis population

In [8]:
%bql GUESS SCHEMA FOR bout_table

,column,stattype,num_distinct,reason
0,Bout Number,nominal,25.0,There are fewer than 20 distinct numerical va...
1,Para Az,numerical,1402.0,There are at least 20 unique numerical values...
2,Para Alt,numerical,1402.0,There are at least 20 unique numerical values...
3,Para Dist,numerical,1402.0,There are at least 20 unique numerical values...
4,Postbout Para Az,numerical,1349.0,There are at least 20 unique numerical values...
5,Postbout Para Alt,numerical,1349.0,There are at least 20 unique numerical values...
6,Postbout Para Dist,numerical,1349.0,There are at least 20 unique numerical values...
7,Strike Or Abort,nominal,2.0,There are fewer than 20 distinct numerical va...


In [10]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Strike Or Abort";)

""


In [ ]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Hunt ID", "Bout Number", "Strike Or Abort", "Inferred", "Percent Nans in Bout", 
                                "Cluster Membership", "Avg Para Velocity", "Postbout Para Az", "Postbout Para Alt", 
                                "Postbout Para Dist";)

### b. Creating and analyzing a probabilistic model (automatically)

In [11]:
%mml CREATE GENERATOR huntbouts_crosscat FOR bout_population;

""


In [12]:
%mml INITIALIZE 50 MODELS IF NOT EXISTS FOR huntbouts_crosscat;

""


In [13]:
%mml ANALYZE huntbouts_crosscat FOR 100 ITERATIONS (OPTIMIZED);
# note you can use the (OPTIMIZED) flag here but that gave you weird results last time. 

Completed: 100 iterations in 11.598669 seconds.
Completed: 100 iterations in 12.005739 seconds.
Completed: 100 iterations in 11.807401 seconds.
Completed: 100 iterations in 13.138245 seconds.
Completed: 100 iterations in 8.086068 seconds.
Completed: 100 iterations in 11.083287 seconds.
Completed: 100 iterations in 11.931882 seconds.
Completed: 100 iterations in 9.494003 seconds.
Completed: 100 iterations in 10.942706 seconds.
Completed: 100 iterations in 11.462989 seconds.
Completed: 100 iterations in 11.090378 seconds.
Completed: 100 iterations in 9.854854 seconds.
Completed: 100 iterations in 8.125150 seconds.
Completed: 100 iterations in 11.570309 seconds.
Completed: 100 iterations in 12.031792 seconds.
Completed: 100 iterations in 11.254563 seconds.
Completed: 100 iterations in 14.141239 seconds.
Completed: 100 iterations in 10.584402 seconds.
Completed: 100 iterations in 10.715074 seconds.
Completed: 100 iterations in 13.218891 seconds.
Completed: 100 iterations in 9.347302 second

""
